In [1]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Analyzing a GEOGRAPHY column with `bigframes.geopandas.GeoSeries`

In [2]:
import bigframes
import bigframes.geopandas
import bigframes.pandas as bpd
bpd.options.display.progress_bar = None

### 1. Load the Counties table from the Census Bureau US Boundaries dataset

In [3]:
df = bpd.read_gbq("bigquery-public-data.geo_us_boundaries.counties")

/usr/local/google/home/arwas/src1/python-bigquery-dataframes/bigframes/session/_io/bigquery/read_gbq_table.py:280: DefaultIndexWarning: Table 'bigquery-public-data.geo_us_boundaries.counties' is clustered and/or partitioned, but BigQuery DataFrames was not able to find a suitable index. To avoid this warning, set at least one of: `index_col` or `filters`.
  warnings.warn(msg, category=bfe.DefaultIndexWarning)


### 2. Create a series from the int_point_geom column

In [4]:
point_geom_series = df['int_point_geom']

## The `GeoSeries` constructor accepts local data or a `bigframes.pandas.Series` object.

### 1. Create a GeoSeries from local data with `Peek`

In [5]:
five_geo_points = point_geom_series.peek(n = 5)
five_geo_points

78        POINT (-95.84727 44.4092)
130      POINT (-94.90431 41.67918)
544      POINT (-95.85272 40.38739)
995     POINT (-101.83333 47.30715)
1036     POINT (-88.36343 37.20952)
Name: int_point_geom, dtype: geometry

### 2. Convert the five geo points to `GeoSeries`

In [6]:
geo_points = bigframes.geopandas.GeoSeries(
        [point for point in five_geo_points]
)
geo_points

0      POINT (-95.84727 44.4092)
1     POINT (-94.90431 41.67918)
2     POINT (-95.85272 40.38739)
3    POINT (-101.83333 47.30715)
4     POINT (-88.36343 37.20952)
dtype: geometry

### 3. Retrieve the x (longitude) and y (latitude) from the GeoSeries with `.x` and `.y`.

#### Note: TypeError is raised if `.x` and `.y` are used with a geometry type other than `Point`.

### `.x`

In [7]:
geo_points.x

0    -95.847268
1    -94.904312
2    -95.852721
3   -101.833328
4    -88.363426
dtype: Float64

### `.y`

In [8]:
geo_points.y

0    44.409195
1    41.679178
2    40.387389
3    47.307147
4    37.209517
dtype: Float64

### 4. Alternatively, use the `.geo` accessor to access GeoSeries methods from a `bigframes.pandas.Series` object.

#### `geo.x`

In [9]:
point_geom_series.geo.x

0    -101.298265
1     -99.111085
2      -66.58687
3    -102.601791
4     -71.578625
5     -88.961529
6     -87.492986
7     -82.422666
8    -100.208166
9     -85.815939
10   -101.681133
11   -119.516659
12    -89.398306
13    -107.78848
14    -91.159306
15   -113.887042
16    -83.470416
17    -98.520146
18    -83.911718
19    -87.321865
20    -91.727626
21    -93.466093
22   -101.143324
23    -78.657634
24    -94.272323
dtype: Float64

#### `geo.y`

In [10]:
point_geom_series.geo.y

0     46.710819
1     29.353661
2     18.211152
3     38.835646
4     41.869768
5     39.860237
6     36.892059
7     38.143642
8     34.524623
9     30.862007
10    40.180165
11    46.228125
12    36.054196
13    38.154731
14    38.761902
15    44.928506
16    30.447232
17    29.448671
18    42.602532
19    34.529776
20    33.957675
21    42.037538
22    29.875285
23    36.299884
24    44.821657
dtype: Float64

## Retrive the `area` of different geometry shapes. 

### 1. Create a geometry collection from local data with `Peek`

In [11]:
geom_series = df["county_geom"].peek(n = 5)
geom_series

59     POLYGON ((-96.92479 43.43217, -96.92477 43.430...
132    POLYGON ((-91.95104 40.05078, -91.95105 40.050...
223    POLYGON ((-84.39719 40.78658, -84.39718 40.783...
328    POLYGON ((-91.80469 31.48623, -91.80469 31.486...
396    POLYGON ((-79.87705 40.03683, -79.87688 40.036...
Name: county_geom, dtype: geometry

### 2. Convert the geometry collection to `GeoSeries`

In [12]:
five_geom = bigframes.geopandas.GeoSeries(
        [point for point in geom_series]
)
five_geom

0    POLYGON ((-96.92479 43.43217, -96.92477 43.430...
1    POLYGON ((-91.95104 40.05078, -91.95105 40.050...
2    POLYGON ((-84.39719 40.78658, -84.39718 40.783...
3    POLYGON ((-91.80469 31.48623, -91.80469 31.486...
4    POLYGON ((-79.87705 40.03683, -79.87688 40.036...
dtype: geometry

## Note: `GeoSeries.area` raises NotImplementedError.  

In [13]:
five_geom.area

NotImplementedError: GeoSeries.area is not supported. Use bigframes.bigquery.st_area(series), instead. Share your usecase with the BigQuery DataFrames team at the https://bit.ly/bigframes-feedback survey.You are currently running BigFrames version 1.36.0

### 3. Use `bigframes.bigquery.st_area` to retirive the `area` in square meters instead. See: https://cloud.google.com/bigquery/docs/reference/standard-sql/geography_functions#st_area

In [14]:
import bigframes.bigquery as bbq

In [15]:
geom_area = bbq.st_area(five_geom)
geom_area

0    1493638545.448335
1    1321524759.411463
2    1052436575.522383
3    1937116615.360128
4    2065462414.544471
dtype: Float64

##  Use `GeoSeries.from_xy()` to create a GeoSeries of `Point` geometries. 

### 1. Reuse the `geo_points.x` and `geo_points.y` results by passing them to `.from_xy()` 

In [16]:
bigframes.geopandas.GeoSeries.from_xy(geo_points.x, geo_points.y)

0      POINT (-95.84727 44.4092)
1     POINT (-94.90431 41.67918)
2     POINT (-95.85272 40.38739)
3    POINT (-101.83333 47.30715)
4     POINT (-88.36343 37.20952)
dtype: geometry

## Use `GeoSeries.to_wkt()` to convert geo points from geometry data type to Well-Knonw Text (WKT).

### 1. Reuse the `geo_points`

In [18]:
geo_to_wkts = bigframes.geopandas.GeoSeries.to_wkt(geo_points)
geo_to_wkts

0     POINT(-95.8472678 44.4091953)
1      POINT(-94.9043119 41.679178)
2     POINT(-95.8527214 40.3873891)
3    POINT(-101.8333279 47.3071473)
4     POINT(-88.3634261 37.2095174)
dtype: string

## Use `GeoSeries.from_wkt()` to convert geo points from Well-Knonw Text (WKT) to geometry data type.

### 1. Reuse `geo_to_wkts` results from `GeoSeries.to_wkts`

In [19]:
wkts_from_geo = bigframes.geopandas.GeoSeries.from_wkt(geo_to_wkts)
wkts_from_geo

0      POINT (-95.84727 44.4092)
1     POINT (-94.90431 41.67918)
2     POINT (-95.85272 40.38739)
3    POINT (-101.83333 47.30715)
4     POINT (-88.36343 37.20952)
dtype: geometry